# Element Load Transformation

## Beam load

This document describes the process of transforming element loads, such as **uniformly distributed loads (UDL)** and **point loads**, from the **global coordinate system** to the **local coordinate system**.

In [16]:
import opstool as opst
import openseespy.opensees as ops

### 2D Case

First we create three 2D beam elements:

In [17]:
ops.wipe()
ops.model("basic", "-ndm", 2, "-ndf", 3)
ops.node(1, 0, 0)
ops.node(2, 0, 2)
ops.node(3, 2, 2)

ops.geomTransf("Linear", 1)
ops.element("elasticBeamColumn", 1, 1, 2, 1000, 10000, 10000, 1)
ops.element("elasticBeamColumn", 2, 2, 3, 1000, 10000, 10000, 1)
ops.element("elasticBeamColumn", 3, 1, 3, 1000, 10000, 10000, 1)

Then, the time series and load pattern are created, followed by the generation of beam element loads using two functions that can easily transform the loads in the global coordinate system to the local coordinate system of each beam element and generate the loads using the [EleLoad Command](https://opensees.berkeley.edu/wiki/index.php/EleLoad_Command) in OpenSees.

In [18]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_beam_uniform_load([1, 2, 3], wy=-2)

ops.pattern("Plain", 2, 1)
opst.pre.transform_beam_point_load([1, 2, 3], py=-3, xl=0.5)

We can check this visually. We can see that our loads are along the global Y axis and they are correctly transformed to each beam element according to their local axes.

In [19]:
fig = opst.vis.plotly.plot_model(
    show_ele_loads=True, show_nodal_loads=True, load_scale=2
)
# fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [20]:
fig.write_html("images/beamLoad2D_plotly.html", full_html=False, include_plotlyjs="cdn")

### 3D Case

In [21]:
ops.wipe()
ops.model("basic", "-ndm", 3, "-ndf", 6)
ops.node(1, 0, 0, 0)
ops.node(2, 0, 2, 0)
ops.node(3, 2, 2, 0)
ops.node(4, 2, 0, 0)


ops.geomTransf("Linear", 1, 0, 0, 1)
ops.element("elasticBeamColumn", 1, 1, 2, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 2, 2, 3, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 3, 3, 4, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 4, 4, 1, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 5, 1, 3, 1000, 1000, 1000, 1000, 1000, 1000, 1)

In [22]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_beam_uniform_load([1, 2, 3, 4, 5], wy=2, wz=-2)

ops.pattern("Plain", 2, 1)
opst.pre.transform_beam_point_load([1, 2, 3, 4, 5], py=2, pz=-3, xl=0.5)

We can check this visually and see that our loads are correctly transformed into the local axes of each beam element.

In [23]:
fig = opst.vis.plotly.plot_model(
    show_ele_loads=True, show_nodal_loads=True, load_scale=2, show_local_axes=True
)
# fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [24]:
fig.write_html("images/beamLoad3D_plotly.html", full_html=False, include_plotlyjs="cdn")

## Surface Load

According to the static equivalence principle, the surface distributed load is equivalent to the node load.

In [25]:
import opstool as opst
import openseespy.opensees as ops

In [28]:
ops.wipe()
# set up a 3D-6DOFs model
ops.model("Basic", "-ndm", 3, "-ndf", 6)
ops.node(1, 0.0, 0.0, 0.0)
ops.node(2, 1.0, 0.0, 0.0)
ops.node(3, 1.0, 1.0, 0.0)
ops.node(4, 0.0, 1.0, 0.0)

ops.node(5, 2.0, 0.0, 0.0)
ops.node(6, 2.0, 1.0, 0.0)

ops.fix(1, *([1] * 6))
ops.fix(4, *([1] * 6))


# create a fiber shell section with 4 layers of material 1
# each layer has a thickness = 0.025
ops.nDMaterial("ElasticIsotropic", 1, 1000.0, 0.2)
ops.section("LayeredShell", 11, 4, 1, 0.025, 1, 0.025, 1, 0.025, 1, 0.025)
# create the shell element using the small displacements/rotations assumption
ops.element("ASDShellQ4", 1, 1, 2, 3, 4, 11)
ops.element("ASDShellT3", 2, 2, 5, 6, 11)
ops.element("ASDShellT3", 3, 6, 3, 2, 11)

In [29]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_surface_uniform_load(ele_tags=[1, 2, 3], p=-1)

Since the surface load intensity is 1 and the total area is 2, the total load on the three elements is 2.0. The sum of the nodal loads assigned to all nodes should equal 2.0.

In [30]:
fig = opst.vis.plotly.plot_model(show_nodal_loads=True, load_scale=2)
fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [ ]:
fig.write_html(
    "images/surfaceLoad3D_plotly.html", full_html=False, include_plotlyjs="cdn"
)

### Analysis and Results

In [31]:
ops.system("BandGeneral")
# Create the constraint handler, the transformation method
ops.constraints("Transformation")
# Create the DOF numberer, the reverse Cuthill-McKee algorithm
ops.numberer("RCM")
# Create the convergence test, the norm of the residual with a tolerance of
# 1e-12 and a max number of iterations of 10
ops.test("NormDispIncr", 1.0e-12, 10, 3)
# Create the solution algorithm, a Newton-Raphson algorithm
ops.algorithm("Newton")
# Create the integration scheme, the LoadControl scheme using steps of 0.1
ops.integrator("LoadControl", 0.1)
# Create the analysis object
ops.analysis("Static")

In [32]:
ODB = opst.post.CreateODB(odb_tag=1)
for i in range(10):
    ops.analyze(1)
    ODB.fetch_response_step()
ODB.save_response()

OPSTOOL ::  All responses data with odb_tag = 1 saved in _OPSTOOL_ODB/RespStepData-1.nc!

In [33]:
node_resp = opst.post.get_nodal_responses(odb_tag=1)
node_resp

OPSTOOL ::  Loading None response data from _OPSTOOL_ODB/RespStepData-1.nc ...

<xarray.Dataset> Size: 20kB
Dimensions:             (time: 11, nodeTags: 6, DOFs: 6)
Coordinates:
  * nodeTags            (nodeTags) int32 24B 1 2 3 4 5 6
  * DOFs                (DOFs) <U2 48B 'UX' 'UY' 'UZ' 'RX' 'RY' 'RZ'
  * time                (time) float64 88B 0.0 0.1 0.2 0.3 ... 0.7 0.8 0.9 1.0
Data variables:
    disp                (time, nodeTags, DOFs) float64 3kB ...
    vel                 (time, nodeTags, DOFs) float64 3kB ...
    accel               (time, nodeTags, DOFs) float64 3kB ...
    reaction            (time, nodeTags, DOFs) float64 3kB ...
    reactionIncInertia  (time, nodeTags, DOFs) float64 3kB ...
    rayleighForces      (time, nodeTags, DOFs) float64 3kB ...
    pressure            (time, nodeTags) float64 528B ...

Retrieve displacement:

In [36]:
node_disp = node_resp["disp"].sel(nodeTags=[5, 6], DOFs="UZ")
node_disp.data

array([[  0.        ,   0.        ],
       [ -2.51381784,  -2.59668341],
       [ -5.02763567,  -5.19336681],
       [ -7.54145351,  -7.79005022],
       [-10.05527135, -10.38673363],
       [-12.56908919, -12.98341704],
       [-15.08290702, -15.58010044],
       [-17.59672486, -18.17678385],
       [-20.1105427 , -20.77346726],
       [-22.62436053, -23.37015066],
       [-25.13817837, -25.96683407]])

Retrieve node reactions:

In [38]:
node_react = node_resp["reaction"].sel(nodeTags=[1, 4], DOFs="UZ")
node_react.data

array([[0.        , 0.        ],
       [0.10629704, 0.09370296],
       [0.21259408, 0.18740592],
       [0.31889112, 0.28110888],
       [0.42518816, 0.37481184],
       [0.5314852 , 0.4685148 ],
       [0.63778224, 0.56221776],
       [0.74407928, 0.65592072],
       [0.85037632, 0.74962368],
       [0.95667335, 0.84332665],
       [1.06297039, 0.93702961]])